In [7]:
import pandas as pd
import networkx as nx
import os

# === CONFIG ===
structures_csv = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\code\cfos_preprocessing\allen_mouse_10um_v1.2\structures.csv"
category_csv = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\pe_grouped.csv"
input_excel = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\mean_dff_per_region_by_mouse_brainmapper.xlsx"
output_file = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\collapsed_region_matrix.xlsx"
target_n_regions = 160

# === LOAD REGION HIERARCHY ===
region_table = pd.read_csv(structures_csv)
region_table['structure_id_path'] = region_table['structure_id_path'].astype(str)

# Build graph from structure_id_path
G = nx.DiGraph()
for _, row in region_table.iterrows():
    path = row["structure_id_path"].strip("/").split("/")
    for i in range(len(path) - 1):
        G.add_edge(path[i], path[i + 1])

# === COLLAPSE TREE TO ~130 REGIONS (STRUCTURE ONLY) ===
def collapse_tree_to_n(graph, n_regions):
    leaves = [node for node in graph.nodes if graph.out_degree(node) == 0]
    all_paths = [nx.shortest_path(graph, source="997", target=leaf) for leaf in leaves if nx.has_path(graph, "997", leaf)]
    from collections import Counter
    counts = Counter()
    for path in all_paths:
        counts.update(path)
    ranked = [node for node, _ in counts.most_common()]
    selected = set()
    for node in ranked:
        descendants = nx.descendants(graph, node)
        if not any(d in selected for d in descendants):
            selected.add(node)
        if len(selected) >= n_regions:
            break
    return list(selected)

collapsed_region_ids = collapse_tree_to_n(G, target_n_regions)


# === MAP ALL STRUCTURE IDs TO THEIR COLLAPSED TARGET REGION ===
structure_to_target = {}
for _, row in region_table.iterrows():
    sid = str(row["id"])
    path = row["structure_id_path"].strip("/").split("/")
    for node in reversed(path):
        if node in collapsed_region_ids:
            structure_to_target[sid] = node
            break

# === LOAD AND MAP DFF DATA FROM ALL MICE ===
df_by_mouse = pd.read_excel(input_excel, sheet_name=None)
matrix_data = {}

for sheet_name, df_mouse in df_by_mouse.items():
    if sheet_name.lower() == "summary" or "structure_id_path" not in df_mouse.columns:
        continue

    region_means = {}
    for _, row in df_mouse.iterrows():
        path = row["structure_id_path"].strip("/").split("/")
        leaf_id = path[-1]
        target = structure_to_target.get(leaf_id)
        if target:
            region_means.setdefault(target, []).append(row["mean_dff"])

    averaged = {rid: sum(vals) / len(vals) for rid, vals in region_means.items()}
    matrix_data[sheet_name] = averaged


# === ASSEMBLE WIDE MATRIX ===
df_matrix = pd.DataFrame(matrix_data).T  # Mice × Regions
available_regions = [rid for rid in collapsed_region_ids if rid in df_matrix.columns]
missing = set(collapsed_region_ids) - set(df_matrix.columns)
print(f"[INFO] {len(missing)} regions had no data and were excluded.")
df_matrix = df_matrix[available_regions]
df_matrix = df_matrix.T  # Regions × Mice

# === ADD REGION INFO: acronym, name, depth ===
id_to_meta = region_table.set_index("id")[["acronym", "name", "structure_id_path"]].astype(str)
df_matrix["acronym"] = df_matrix.index.map(lambda x: id_to_meta.loc[int(x), "acronym"] if int(x) in id_to_meta.index else "")
df_matrix["name"] = df_matrix.index.map(lambda x: id_to_meta.loc[int(x), "name"] if int(x) in id_to_meta.index else "")
df_matrix["depth"] = df_matrix.index.map(lambda x: len(id_to_meta.loc[int(x), "structure_id_path"].strip("/").split("/")) if int(x) in id_to_meta.index else None)
df_matrix.insert(0, "region_id", df_matrix.index)

# === ADD CATEGORY FROM pe_grouped.csv ===
category_df = pd.read_csv(category_csv)
category_df["abbrev"] = category_df["abbrev"].astype(str)
category_df["name"] = category_df["name"].astype(str)

df_matrix["category"] = df_matrix["acronym"].map(dict(zip(category_df["abbrev"], category_df["category"])))
unmatched = df_matrix["category"].isna()
df_matrix.loc[unmatched, "category"] = df_matrix.loc[unmatched, "name"].map(
    dict(zip(category_df["name"], category_df["category"]))
)

# === SAVE ===
df_matrix.to_excel(output_file, index=False)
print(f"Final matrix saved to:\n{output_file}")

[INFO] 47 regions had no data and were excluded.
Final matrix saved to:
Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\collapsed_region_matrix.xlsx


In [ ]:
354	Medulla	/997/8/343/1065/354/
771	Pons	/997/8/343/1065/771/
1097	Hypothalamus	/997/8/343/1129/1097/
549	Thalamus	/997/8/343/1129/549/
313	Midbrain	/997/8/343/313/
512	Cerebellum	/997/8/512/
695	Cortical plate	/997/8/567/688/695/
703	Cortical subplate	/997/8/567/688/703/
803	Pallidum	/997/8/567/623/803/
477	Striatum	/997/8/567/623/477/


In [10]:
import pandas as pd
import networkx as nx
import os

# === CONFIG ===
structures_csv = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\code\cfos_preprocessing\allen_mouse_10um_v1.2\structures.csv"
input_excel = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\mean_dff_per_region_by_mouse_brainmapper.xlsx"
output_file = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\colapsed_dff_cleaned_210.xlsx"
target_n_regions = 210

# === LOAD REGION HIERARCHY ===
region_table = pd.read_csv(structures_csv)
region_table['structure_id_path'] = region_table['structure_id_path'].astype(str)
id_to_path = region_table.set_index("id")["structure_id_path"].astype(str).to_dict()

# Build anatomical graph
G = nx.DiGraph()
for _, row in region_table.iterrows():
    path = row["structure_id_path"].strip("/").split("/")
    for i in range(len(path) - 1):
        G.add_edge(path[i], path[i + 1])

# Collapse to 130 nodes
def collapse_tree_to_n(graph, n_regions):
    leaves = [node for node in graph.nodes if graph.out_degree(node) == 0]
    all_paths = [nx.shortest_path(graph, source="997", target=leaf) for leaf in leaves if nx.has_path(graph, "997", leaf)]
    from collections import Counter
    counts = Counter()
    for path in all_paths:
        counts.update(path)
    ranked = [node for node, _ in counts.most_common()]
    selected = set()
    for node in ranked:
        descendants = nx.descendants(graph, node)
        if not any(d in selected for d in descendants):
            selected.add(node)
        if len(selected) >= n_regions:
            break
    return list(selected)

collapsed_region_ids = collapse_tree_to_n(G, target_n_regions)

# === REMOVE REDUNDANT PARENT REGIONS ===
collapsed_with_paths = [(rid, id_to_path.get(int(rid), "")) for rid in collapsed_region_ids]

def is_subpath(p1, p2):
    return p2.startswith(p1) and p1 != p2

filtered_regions = []
paths = [p for _, p in collapsed_with_paths]
for i, (rid1, path1) in enumerate(collapsed_with_paths):
    if any(is_subpath(path1, p2) for j, p2 in enumerate(paths) if j != i):
        continue
    filtered_regions.append(rid1)

collapsed_region_ids = filtered_regions
print(f"[INFO] Reduced to {len(collapsed_region_ids)} non-overlapping regions.")

# === MAP STRUCTURE TO COLLAPSED REGIONS ===
structure_to_target = {}
for _, row in region_table.iterrows():
    sid = str(row["id"])
    path = row["structure_id_path"].strip("/").split("/")
    for node in reversed(path):
        if node in collapsed_region_ids:
            structure_to_target[sid] = node
            break

# === LOAD MOUSE DFF ===
df_by_mouse = pd.read_excel(input_excel, sheet_name=None)
matrix_data = {}

for sheet_name, df_mouse in df_by_mouse.items():
    if sheet_name.lower() == "summary" or "structure_id_path" not in df_mouse.columns:
        continue
    region_means = {}
    for _, row in df_mouse.iterrows():
        path = row["structure_id_path"].strip("/").split("/")
        leaf_id = path[-1]
        target = structure_to_target.get(leaf_id)
        if target:
            region_means.setdefault(target, []).append(row["mean_dff"])
    averaged = {rid: sum(vals) / len(vals) for rid, vals in region_means.items()}
    matrix_data[sheet_name] = averaged

# === FINAL MATRIX ===
df_matrix = pd.DataFrame(matrix_data).T
available_regions = [rid for rid in collapsed_region_ids if rid in df_matrix.columns]
missing = set(collapsed_region_ids) - set(df_matrix.columns)
print(f"[INFO] {len(missing)} collapsed regions had no data.")

df_matrix = df_matrix[available_regions].T
df_matrix["region_id"] = df_matrix.index.astype(int)

# Add structure metadata
id_meta = region_table.set_index("id")[["acronym", "name", "structure_id_path"]].astype(str)
df_matrix["acronym"] = df_matrix["region_id"].map(lambda x: id_meta.loc[x, "acronym"] if x in id_meta.index else "")
df_matrix["name"] = df_matrix["region_id"].map(lambda x: id_meta.loc[x, "name"] if x in id_meta.index else "")
df_matrix["structure_id_path"] = df_matrix["region_id"].map(lambda x: id_meta.loc[x, "structure_id_path"] if x in id_meta.index else "")
df_matrix["depth"] = df_matrix["structure_id_path"].map(lambda p: len(p.strip("/").split("/")))

# Add anatomical category based on path prefix
category_map = {
    "Medulla": "/997/8/343/1065/354/",
    "Pons": "/997/8/343/1065/771/",
    "Hypothalamus": "/997/8/343/1129/1097/",
    "Thalamus": "/997/8/343/1129/549/",
    "Midbrain": "/997/8/343/313/",
    "Cerebellum": "/997/8/512/",
    "Cortical plate": "/997/8/567/688/695/",
    "Cortical subplate": "/997/8/567/688/703/",
    "Pallidum": "/997/8/567/623/803/",
    "Striatum": "/997/8/567/623/477/"
}

def assign_category(path):
    for cat, prefix in category_map.items():
        if path.startswith(prefix):
            return cat
    return "Other"

df_matrix["category"] = df_matrix["structure_id_path"].map(assign_category)

# Reorder columns
meta_cols = ["region_id", "acronym", "name", "structure_id_path", "depth", "category"]
df_matrix = df_matrix[meta_cols + [col for col in df_matrix.columns if col not in meta_cols]]

# === SAVE ===
df_matrix.to_excel(output_file, index=False)
print(f" Final matrix saved to:\n{output_file}")

[INFO] Reduced to 134 non-overlapping regions.
[INFO] 18 collapsed regions had no data.
 Final matrix saved to:
Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\colapsed_dff_cleaned_210.xlsx


In [12]:
import pandas as pd

# === CONFIG ===
structures_path = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\code\cfos_preprocessing\allen_mouse_10um_v1.2\structures.csv"
output_csv = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\child_to_collapsed_map.csv"

# === Load region hierarchy ===
region_table = pd.read_csv(structures_path)
region_table["structure_id_path"] = region_table["structure_id_path"].astype(str)

id_to_path = region_table.set_index("id")["structure_id_path"].astype(str).to_dict()
id_to_name = region_table.set_index("id")["name"].astype(str).to_dict()

# === Define the 10 collapsed categories (based on root IDs) ===
collapsed_region_ids = [
    354,   # Medulla
    771,   # Pons
    1097,  # Hypothalamus
    549,   # Thalamus
    313,   # Midbrain
    512,   # Cerebellum
    695,   # Cortical plate
    703,   # Cortical subplate
    803,   # Pallidum
    477    # Striatum
]
collapsed_region_ids = list(map(str, collapsed_region_ids))

# === Build structure_id_path mapping ===
collapsed_paths = {cid: id_to_path[int(cid)] for cid in collapsed_region_ids}

# === Map each region to its collapsed parent ===
mapping_rows = []
for sid, path in id_to_path.items():
    assigned_parent = None
    for parent_id, parent_path in collapsed_paths.items():
        parent_path_clean = parent_path.strip("/")
        if path.startswith(parent_path_clean + "/") or path == parent_path_clean:
            assigned_parent = parent_id
            break

    if assigned_parent:
        mapping_rows.append({
            "child_region_id": sid,
            "child_region_name": id_to_name.get(int(sid), ""),
            "child_structure_id_path": path,
            "collapsed_region_id": assigned_parent,
            "collapsed_region_name": id_to_name.get(int(assigned_parent), "")
        })

df_map = pd.DataFrame(mapping_rows)

# === Save or display ===
print(f"Mapped {len(df_map)} regions to collapsed parents.")
print(df_map.head(10))

df_map.to_csv(output_csv, index=False)
print(f"Saved to: {output_csv}")


Mapped 0 regions to collapsed parents.
Empty DataFrame
Columns: []
Index: []
Saved to: Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\child_to_collapsed_map.csv


In [15]:
import pandas as pd
import networkx as nx
import os
from collections import defaultdict

# === CONFIG ===
structures_path = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\code\cfos_preprocessing\allen_mouse_10um_v1.2\structures.csv"
input_excel = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\mean_dff_per_region_by_mouse_brainmapper.xlsx"
output_matrix = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\final_collapsed_matrix.xlsx"
output_log = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\dff_collapsing_log.csv"
TARGET_N = 160

# === Load hierarchy ===
region_table = pd.read_csv(structures_path)
region_table["structure_id_path"] = region_table["structure_id_path"].astype(str)
id_to_path = region_table.set_index("id")["structure_id_path"].astype(str).to_dict()
id_to_name = region_table.set_index("id")["name"].astype(str).to_dict()

# === Build graph from structure_id_path ===
G = nx.DiGraph()
for _, row in region_table.iterrows():
    path = row["structure_id_path"].strip("/").split("/")
    for i in range(len(path) - 1):
        G.add_edge(path[i], path[i + 1])

# === Collapse function ===
def collapse_tree_to_n(graph, n):
    leaves = [n for n in graph.nodes if graph.out_degree(n) == 0]
    all_paths = [nx.shortest_path(graph, source="997", target=leaf)
                 for leaf in leaves if nx.has_path(graph, "997", leaf)]
    from collections import Counter
    counts = Counter()
    for path in all_paths:
        counts.update(path)
    ranked = [node for node, _ in counts.most_common()]
    selected = set()
    for node in ranked:
        descendants = nx.descendants(graph, node)
        if not any(d in selected for d in descendants):
            selected.add(node)
        if len(selected) >= n:
            break
    return list(selected)

collapsed_region_ids = collapse_tree_to_n(G, TARGET_N)

# === Map child → parent ===
structure_to_collapsed = {}
for sid, path in id_to_path.items():
    parts = path.strip("/").split("/")
    for node in reversed(parts):
        if node in collapsed_region_ids:
            structure_to_collapsed[str(sid)] = node
            break

# === Load per-mouse DFF ===
df_by_mouse = pd.read_excel(input_excel, sheet_name=None)
matrix_data = defaultdict(dict)
log_rows = []

for sheet_name, df_mouse in df_by_mouse.items():
    if sheet_name.lower() == "summary" or "structure_id_path" not in df_mouse.columns:
        continue
    mouse_id = sheet_name
    temp_assignments = defaultdict(list)

    for _, row in df_mouse.iterrows():
        path = row["structure_id_path"].strip("/").split("/")
        region_id = path[-1]
        collapsed_id = structure_to_collapsed.get(region_id)
        if collapsed_id:
            temp_assignments[collapsed_id].append((region_id, row["mean_dff"]))

            # Logging for traceability
            log_rows.append({
                "mouse": mouse_id,
                "collapsed_region_id": collapsed_id,
                "collapsed_region_name": id_to_name.get(int(collapsed_id), ""),
                "child_region_id": region_id,
                "child_region_name": id_to_name.get(int(region_id), ""),
                "child_structure_id_path": row["structure_id_path"],
                "mean_dff_contribution": row["mean_dff"]
            })

    for cid, values in temp_assignments.items():
        mean_val = sum(v for _, v in values) / len(values)
        matrix_data[mouse_id][cid] = mean_val

# === Build matrix
df_matrix = pd.DataFrame(matrix_data).T
df_matrix.index.name = "mouse"
df_matrix.columns.name = "collapsed_region_id"
df_matrix = df_matrix.T
df_matrix["region_id"] = df_matrix.index.astype(int)
df_matrix["name"] = df_matrix["region_id"].map(lambda x: id_to_name.get(x, ""))
df_matrix["structure_id_path"] = df_matrix["region_id"].map(lambda x: id_to_path.get(x, ""))
df_matrix["depth"] = df_matrix["structure_id_path"].map(lambda p: len(p.strip("/").split("/")))

# Move metadata columns to front
meta_cols = ["region_id", "name", "structure_id_path", "depth"]
df_matrix = df_matrix[meta_cols + [col for col in df_matrix.columns if col not in meta_cols]]

# === Save
df_matrix.to_excel(output_matrix, index=False)
print(f"Collapsed matrix saved to:\n{output_matrix}")

log_df = pd.DataFrame(log_rows)
log_df.to_csv(output_log, index=False)
print(f"Collapse log with ΔF/F contributions saved to:\n{output_log}")


Collapsed matrix saved to:
Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\final_collapsed_matrix.xlsx
Collapse log with ΔF/F contributions saved to:
Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\dff_collapsing_log.csv


In [17]:
import pandas as pd
import networkx as nx
import os
from collections import defaultdict
import numpy as np

# === CONFIG ===
structures_path = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\code\cfos_preprocessing\allen_mouse_10um_v1.2\structures.csv"
input_excel = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\mean_dff_per_region_by_mouse_brainmapper.xlsx"
output_excel = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\final_collapsed_matrix_2.xlsx"
output_log = r"Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\dff_collapsing_log_2.csv"
TARGET_N = 160
# === Load Allen structure data ===
region_table = pd.read_csv(structures_path)
region_table["structure_id_path"] = region_table["structure_id_path"].astype(str)
id_to_acronym = region_table.set_index("id")["acronym"].astype(str).to_dict()
id_to_path = region_table.set_index("id")["structure_id_path"].astype(str).to_dict()
id_to_name = region_table.set_index("id")["name"].astype(str).to_dict()

# === Build anatomical hierarchy graph ===
G = nx.DiGraph()
for _, row in region_table.iterrows():
    path = row["structure_id_path"].strip("/").split("/")
    for i in range(len(path) - 1):
        G.add_edge(path[i], path[i + 1])

# === Collapse region selection ===
def collapse_tree_to_n(graph, n):
    leaves = [n for n in graph.nodes if graph.out_degree(n) == 0]
    all_paths = [nx.shortest_path(graph, source="997", target=leaf)
                 for leaf in leaves if nx.has_path(graph, "997", leaf)]
    from collections import Counter
    counts = Counter()
    for path in all_paths:
        counts.update(path)
    ranked = [node for node, _ in counts.most_common()]
    selected = set()
    for node in ranked:
        descendants = nx.descendants(graph, node)
        if not any(d in selected for d in descendants):
            selected.add(node)
        if len(selected) >= n:
            break
    return list(selected)

collapsed_region_ids = collapse_tree_to_n(G, TARGET_N)

# === Map child → collapsed parent ===
structure_to_collapsed = {}
for sid, path in id_to_path.items():
    parts = path.strip("/").split("/")
    for node in reversed(parts):
        if node in collapsed_region_ids:
            structure_to_collapsed[str(sid)] = node
            break

# === Process all mice ===
df_by_mouse = pd.read_excel(input_excel, sheet_name=None)
mean_data = defaultdict(dict)
std_data = defaultdict(dict)
sem_data = defaultdict(dict)
log_rows = []

for sheet_name, df_mouse in df_by_mouse.items():
    if sheet_name.lower() == "summary" or "structure_id_path" not in df_mouse.columns:
        continue
    mouse_id = sheet_name
    temp_assignments = defaultdict(list)

    for _, row in df_mouse.iterrows():
        path = row["structure_id_path"].strip("/").split("/")
        region_id = path[-1]
        collapsed_id = structure_to_collapsed.get(region_id)
        if collapsed_id:
            temp_assignments[collapsed_id].append(row["mean_dff"])
            log_rows.append({
                "mouse": mouse_id,
                "collapsed_region_id": collapsed_id,
                "collapsed_region_name": id_to_name.get(int(collapsed_id), ""),
                "child_region_id": region_id,
                "child_region_name": id_to_name.get(int(region_id), ""),
                "child_structure_id_path": row["structure_id_path"],
                "mean_dff_contribution": row["mean_dff"]
            })

    for cid, values in temp_assignments.items():
        arr = np.array(values, dtype=float)
        mean_data[mouse_id][cid] = np.mean(arr)
        std_data[mouse_id][cid] = np.std(arr, ddof=1) if len(arr) > 1 else 0.0
        sem_data[mouse_id][cid] = std_data[mouse_id][cid] / np.sqrt(len(arr)) if len(arr) > 1 else 0.0

# === Format outputs
def build_df(data_dict, id_to_name, id_to_path, id_to_acronym):
    df = pd.DataFrame(data_dict).T
    df.index.name = "mouse"
    df.columns.name = "collapsed_region_id"
    df = df.T
    df["region_id"] = df.index.astype(int)
    df["name"] = df["region_id"].map(lambda x: id_to_name.get(x, ""))
    df["acronym"] = df["region_id"].map(lambda x: id_to_acronym.get(x, ""))
    df["structure_id_path"] = df["region_id"].map(lambda x: id_to_path.get(x, ""))
    df["depth"] = df["structure_id_path"].map(lambda p: len(p.strip("/").split("/")))
    meta_cols = ["region_id", "name", "acronym", "structure_id_path", "depth"]
    return df[meta_cols + [col for col in df.columns if col not in meta_cols]]

df_mean = build_df(mean_data, id_to_name, id_to_path, id_to_acronym)
df_std = build_df(std_data, id_to_name, id_to_path, id_to_acronym)
df_sem = build_df(sem_data, id_to_name, id_to_path, id_to_acronym)

# === Export all
with pd.ExcelWriter(output_excel) as writer:
    df_mean.to_excel(writer, sheet_name="mean_dff", index=False)
    df_std.to_excel(writer, sheet_name="std_dff", index=False)
    df_sem.to_excel(writer, sheet_name="sem_dff", index=False)
print(f"Matrix with mean, std, sem saved to:\n{output_excel}")

# Save log
pd.DataFrame(log_rows).to_csv(output_log, index=False)
print(f"Detailed ΔF/F log saved to:\n{output_log}")


Matrix with mean, std, sem saved to:
Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\final_collapsed_matrix_2.xlsx
Detailed ΔF/F log saved to:
Y:\public\projects\AnAl_20240405_Neuromod_PE\PE_mapping\processed_data\dff_downsampled\dff_collapsing_log_2.csv
